# Comparaison : Content Understanding vs ARGUS

Comparaison de deux approches d'extraction documentaire appliquées au même batch de **20 factures** (batch1).

## Méthodes comparées
| | Méthode 1 – Content Understanding | Méthode 2 – ARGUS |
|---|---|---|
| **Repo** | Notre pipeline (`docu_prebuilt_invoice.ipynb`) | [Azure-Samples/ARGUS](https://github.com/Azure-Samples/ARGUS) |
| **Approche** | API pré-entraînée `prebuilt-invoice` | OCR + GPT-4 Vision (hybrid pipeline) |
| **Philosophie** | Extraction directe, schéma fixe | Extraction flexible, schéma personnalisable |

## 1. Comparaison des Architectures

| Critère | Content Understanding (Méthode 1) | ARGUS (Méthode 2) |
|---|---|---|
| **OCR / Extraction** | Modèle pré-entraîné `prebuilt-invoice` (Content Understanding API v2025-11-01) | Azure Document Intelligence `prebuilt-layout` → texte markdown |
| **Structuration** | 31 champs invoice extraits automatiquement par le modèle | GPT-4 Vision reçoit OCR text + images → JSON libre selon schema |
| **Schéma** | Fixe (31 champs Microsoft prédéfinis) | Configurable par dataset (prompt + schema personnalisé) |
| **Rôle du LLM** | GPT-4.1 Vision pour description textuelle uniquement (optionnel) | GPT-4 Vision pour extraction structurée + évaluation + résumé |
| **Pipeline** | Upload Blob → SAS URL → Content Understanding → (LLM description) | Upload Blob → Event Grid → OCR → GPT Extraction → GPT Evaluation → GPT Summary |
| **Stockage résultats** | Fichiers JSON locaux | Azure Cosmos DB + Blob Storage |
| **Infrastructure** | Notebook Python + API REST | FastAPI + Container Apps + Logic Apps + Cosmos DB |
| **Évaluation qualité** | Confidence scores natifs du modèle (par champ) | GPT évalue chaque champ extrait avec confidence scores + accuracy |
| **Coût estimé/doc** | ~$0.01-0.02 (CU) + ~$0.03 (LLM opt.) | ~$0.01 (DI) + ~$0.10-0.30 (3× GPT-4 Vision) |
| **Flexibilité schéma** | ❌ Fixe | ✅ Totalement configurable |
| **Auto-évaluation** | ❌ | ✅ Confidence scores + accuracy % |
| **Chat avec documents** | ❌ | ✅ Endpoint `/api/chat` intégré |

## 2. Pipeline Détaillé

### Content Understanding (notre approche)
```
Image JPG
  → Upload Azure Blob Storage (SAS URL)
  → Content Understanding API (prebuilt-invoice, v2025-11-01)
      → markdown structuré + 31 champs typés (automatique, 1 appel)
  → GPT-4.1 Vision (optionnel)
      → description textuelle du document
  → Sauvegarde JSON local
```

### ARGUS (Azure-Samples/ARGUS)
```
PDF/Image
  → Upload Azure Blob Storage
  → Event Grid trigger → FastAPI Backend (Container Apps)
  → Étape 1: Azure Document Intelligence (prebuilt-layout)
      → OCR markdown (texte brut structuré)
  → Étape 2: PyMuPDF → conversion PDF en images PNG (base64)
  → Étape 3: GPT-4 Vision (OCR text + images + system prompt + output schema)
      → JSON structuré selon le schéma personnalisé
  → Étape 4: GPT-4 Vision (évaluation)
      → confidence scores (0-1) par champ + accuracy globale
  → Étape 5: GPT (résumé)
      → résumé textuel du document
  → Stockage Cosmos DB (document complet + metadata)
```

In [8]:
# === Section 1: Import des bibliothèques ===
import json
import os
import glob
import statistics
import time
import base64
import requests
from pathlib import Path
from collections import Counter
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

print("✅ Bibliothèques chargées")
print(f"   DOC_INTELLIGENCE_ENDPOINT: {os.getenv('DOC_INTELLIGENCE_ENDPOINT', '❌ manquant')[:50]}...")
print(f"   GPT_ENDPOINT: {os.getenv('GPT_ENDPOINT', '❌ manquant')[:60]}...")

✅ Bibliothèques chargées
   DOC_INTELLIGENCE_ENDPOINT: https://docintel-argus-test.cognitiveservices.azur...
   GPT_ENDPOINT: https://content-understanding--resource.cognitiveservices.az...


In [9]:
# === Section 2: Charger les résultats Content Understanding ===
CU_RESULTS_DIR = r"c:\Users\ayac\Downloads\archive (1)\batch_1\batch_1\docu_results_batch1_1\prebuilt-invoice"
cu_files = sorted(glob.glob(os.path.join(CU_RESULTS_DIR, "batch1-*.json")))

cu_metrics = []
for f in cu_files:
    with open(f, encoding="utf-8") as fp:
        data = json.load(fp)

    fname = os.path.basename(f)
    contents = data.get("result", {}).get("contents", [{}])
    fields = contents[0].get("fields", {}) if contents else {}
    markdown = contents[0].get("markdown", "") if contents else ""

    # Compter les champs avec valeur réelle
    fields_with_value = 0
    field_names_with_value = []
    for k, v in fields.items():
        has_val = False
        ftype = v.get("type", "")
        if ftype == "object":
            vo = v.get("valueObject", {})
            for sk, sv in vo.items():
                if isinstance(sv, dict):
                    for vkey in ["valueString", "valueNumber", "valueDate"]:
                        if sv.get(vkey) is not None:
                            has_val = True
                            break
        elif ftype == "array":
            arr = v.get("valueArray", [])
            if arr and len(arr) > 0:
                has_val = True
        else:
            for vkey in ["valueString", "valueNumber", "valueDate"]:
                if v.get(vkey) is not None:
                    has_val = True
                    break
        if has_val:
            fields_with_value += 1
            field_names_with_value.append(k)

    # Confidence scores
    confidences = []
    for k, v in fields.items():
        c = v.get("confidence")
        if c is not None:
            confidences.append(c)
        vo = v.get("valueObject", {})
        for sk, sv in vo.items():
            if isinstance(sv, dict) and sv.get("confidence") is not None:
                confidences.append(sv["confidence"])

    # LLM description
    extracted = data.get("_extracted", {})
    desc = extracted.get("description", "")
    llm_ok = bool(desc) and "error" not in str(desc).lower()[:80]

    cu_metrics.append({
        "file": fname,
        "fields_with_value": fields_with_value,
        "total_fields": len(fields),
        "field_names": field_names_with_value,
        "markdown_chars": len(markdown),
        "avg_confidence": statistics.mean(confidences) if confidences else 0,
        "min_confidence": min(confidences) if confidences else 0,
        "llm_description": "✅" if llm_ok else "❌",
    })

print(f"📄 Content Understanding: {len(cu_metrics)} documents chargés\n")
print(f"{'Document':<20} {'Champs':>8} {'Markdown':>10} {'Conf.moy':>10} {'LLM':>5}")
print("-" * 58)
for m in cu_metrics:
    print(f"{m['file']:<20} {m['fields_with_value']:>3}/{m['total_fields']:<4} {m['markdown_chars']:>10,} {m['avg_confidence']:>9.3f} {m['llm_description']:>5}")

📄 Content Understanding: 20 documents chargés

Document               Champs   Markdown   Conf.moy   LLM
----------------------------------------------------------
batch1-0001.json      17/31        2,011     0.820     ✅
batch1-0002.json      17/31        1,663     0.830     ❌
batch1-0003.json      17/31        1,339     0.825     ❌
batch1-0004.json      17/31        1,697     0.819     ❌
batch1-0005.json      17/31        1,636     0.809     ❌
batch1-0006.json       0/31        1,507     0.720     ❌
batch1-0007.json      17/31        2,060     0.815     ✅
batch1-0008.json      17/31        1,521     0.825     ✅
batch1-0009.json      17/31        1,029     0.822     ❌
batch1-0010.json      17/31        1,093     0.816     ❌
batch1-0011.json      17/31        1,076     0.815     ❌
batch1-0012.json      17/31        1,390     0.814     ✅
batch1-0013.json      17/31        1,529     0.813     ✅
batch1-0014.json      17/31          887     0.811     ✅
batch1-0015.json      17/31        1,7

In [10]:
# === Section 3: Métriques agrégées Content Understanding ===
cu_fields = [m["fields_with_value"] for m in cu_metrics]
cu_markdown = [m["markdown_chars"] for m in cu_metrics]
cu_confs = [m["avg_confidence"] for m in cu_metrics if m["avg_confidence"] > 0]
cu_llm_ok = sum(1 for m in cu_metrics if m["llm_description"] == "✅")

print("=" * 60)
print("   MÉTRIQUES AGRÉGÉES — CONTENT UNDERSTANDING")
print("=" * 60)
print(f"  Documents traités      : {len(cu_metrics)}")
print(f"  Champs/doc (moyenne)   : {statistics.mean(cu_fields):.1f} / {cu_metrics[0]['total_fields']}")
print(f"  Champs/doc (min/max)   : {min(cu_fields)} / {max(cu_fields)}")
print(f"  Docs sans champs (=0)  : {sum(1 for f in cu_fields if f == 0)}")
print(f"  Markdown chars (moy)   : {statistics.mean(cu_markdown):,.0f}")
print(f"  Confidence moyenne     : {statistics.mean(cu_confs):.3f}" if cu_confs else "  Confidence moyenne     : N/A")
print(f"  Confidence min         : {min(cu_confs):.3f}" if cu_confs else "  Confidence min         : N/A")
print(f"  LLM description OK     : {cu_llm_ok}/{len(cu_metrics)}")
print(f"  Taux extraction        : {sum(1 for f in cu_fields if f > 0)}/{len(cu_metrics)} ({sum(1 for f in cu_fields if f > 0)/len(cu_metrics)*100:.0f}%)")

# Champs les plus extraits
all_field_names = []
for m in cu_metrics:
    all_field_names.extend(m["field_names"])
field_counts = Counter(all_field_names)
print(f"\n  Top 10 champs extraits:")
for name, count in field_counts.most_common(10):
    print(f"    {name:<30} → {count}/{len(cu_metrics)} docs")

   MÉTRIQUES AGRÉGÉES — CONTENT UNDERSTANDING
  Documents traités      : 20
  Champs/doc (moyenne)   : 16.1 / 31
  Champs/doc (min/max)   : 0 / 17
  Docs sans champs (=0)  : 1
  Markdown chars (moy)   : 1,506
  Confidence moyenne     : 0.814
  Confidence min         : 0.720
  LLM description OK     : 12/20
  Taux extraction        : 19/20 (95%)

  Top 10 champs extraits:
    AmountDue                      → 19/20 docs
    CountryRegion                  → 19/20 docs
    CustomerAddress                → 19/20 docs
    CustomerAddressRecipient       → 19/20 docs
    CustomerName                   → 19/20 docs
    CustomerTaxId                  → 19/20 docs
    InvoiceDate                    → 19/20 docs
    InvoiceId                      → 19/20 docs
    LineItems                      → 19/20 docs
    SubtotalAmount                 → 19/20 docs


## 3. Pipeline ARGUS-style (exécution réelle)

On **exécute** le pipeline ARGUS sur nos mêmes 20 factures en répliquant ses étapes clés :
1. **OCR Option A** : Azure Document Intelligence `prebuilt-layout` (texte markdown)
2. **OCR Option B** : Mistral Doc AI (OCR alternatif)
3. **GPT-5-chat** pour l'extraction structurée (OCR text + images + prompt ARGUS + schema)

> **Services utilisés** :
> - Document Intelligence : `docintel-argus-test` (API key auth)
> - Mistral Doc AI : `content-understanding--resource` (API key auth)
> - GPT-5-chat : `content-understanding--resource` (API key auth)
> - Credentials chargées depuis `.env`

In [ ]:
# === Section 4: Implémentation du pipeline ARGUS-style ===
# Pipeline : OCR (Doc Intelligence OU Mistral Doc AI) → GPT-5-chat extraction

# --- Credentials from .env ---
DI_ENDPOINT = os.environ["DOC_INTELLIGENCE_ENDPOINT"].rstrip("/")
DI_KEY = os.environ["DOC_INTELLIGENCE_KEY"]

MISTRAL_ENDPOINT = os.environ["MISTRAL_DOC_AI_ENDPOINT"]
MISTRAL_KEY = os.environ["MISTRAL_DOC_AI_KEY"]

# GPT-5-chat : key auth désactivée → utiliser Bearer token (Entra ID)
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

GPT_BASE_URL = "https://content-understanding--resource.cognitiveservices.azure.com"
GPT_DEPLOYMENT = "gpt-5-chat"
GPT_API_VERSION = "2025-01-01-preview"
GPT_FULL_URL = f"{GPT_BASE_URL}/openai/deployments/{GPT_DEPLOYMENT}/chat/completions?api-version={GPT_API_VERSION}"

def get_bearer_token():
    """Obtenir un Bearer token pour Azure AI Services (Entra ID auth)."""
    return credential.get_token("https://cognitiveservices.azure.com/.default").token

# =====================================================================
#  OCR Option A : Azure Document Intelligence (prebuilt-layout)
# =====================================================================
def get_di_ocr_markdown(image_path: str) -> str:
    """OCR via Azure Document Intelligence prebuilt-layout → markdown"""
    with open(image_path, "rb") as f:
        image_data = f.read()

    mime = "image/jpeg" if image_path.lower().endswith((".jpg", ".jpeg")) else "image/png"

    resp = requests.post(
        f"{DI_ENDPOINT}/formrecognizer/documentModels/prebuilt-layout:analyze?api-version=2023-07-31&outputContentFormat=markdown",
        headers={
            "Ocp-Apim-Subscription-Key": DI_KEY,
            "Content-Type": mime,
        },
        data=image_data,
        timeout=60,
    )
    resp.raise_for_status()
    op_url = resp.headers["Operation-Location"]

    for _ in range(60):
        time.sleep(2)
        r = requests.get(op_url, headers={"Ocp-Apim-Subscription-Key": DI_KEY}, timeout=30)
        r.raise_for_status()
        result = r.json()
        if result["status"] == "succeeded":
            return result.get("analyzeResult", {}).get("content", "")
        elif result["status"] == "failed":
            return f"ERROR: {result}"
    return "ERROR: timeout"


# =====================================================================
#  OCR Option B : Mistral Document AI (mistral-document-ai-2505 via Azure)
#  - Auth: Bearer token (Entra ID) car key auth désactivée sur cette ressource
#  - API: /providers/mistral/azure/ocr
#  - Format: type="document_url", document_url="data:<mime>;base64,..."
# =====================================================================
def get_mistral_ocr_markdown(image_path: str) -> str:
    """OCR via Mistral Document AI (Azure-hosted) → markdown"""
    with open(image_path, "rb") as f:
        image_data = f.read()

    b64_image = base64.b64encode(image_data).decode()
    mime = "image/jpeg" if image_path.lower().endswith((".jpg", ".jpeg")) else "image/png"

    payload = {
        "model": "mistral-document-ai-2505",
        "document": {
            "type": "document_url",
            "document_url": f"data:{mime};base64,{b64_image}"
        }
    }

    token = get_bearer_token()
    resp = requests.post(
        MISTRAL_ENDPOINT,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json",
        },
        json=payload,
        timeout=120,
    )
    resp.raise_for_status()
    result = resp.json()

    pages = result.get("pages", [])
    markdown_parts = [p.get("markdown", "") for p in pages]
    return "\n\n".join(markdown_parts)


# =====================================================================
#  GPT-5-chat extraction (Bearer token auth, pattern ARGUS)
# =====================================================================
ARGUS_SYSTEM_PROMPT = """Extract all data from the document in a comprehensive and structured manner.

Focus on:
- Key identifiers (invoice numbers, reference numbers, IDs)
- Financial information (amounts, totals, currency, taxes)
- Parties involved (vendors, customers, suppliers, recipients)
- Dates and timelines (invoice dates, due dates, service periods)
- Line items and details (products, services, quantities, prices)
- Contact information (addresses, phone numbers, emails)
- Any other relevant structured data visible in the document

When both text and images are available, use the text as the primary source and cross-reference with images for accuracy."""

ARGUS_SCHEMA = {
    "invoice_number": "string",
    "invoice_date": "string",
    "due_date": "string",
    "seller_name": "string",
    "seller_address": "string",
    "seller_tax_id": "string",
    "buyer_name": "string",
    "buyer_address": "string",
    "buyer_tax_id": "string",
    "iban": "string",
    "line_items": [{"description": "string", "quantity": "number", "unit_price": "number", "net_worth": "number", "vat_rate": "string", "gross_worth": "number"}],
    "subtotal": "number",
    "total_tax": "number",
    "total_amount": "number",
    "currency": "string"
}


def gpt_extract_argus_style(ocr_text: str, image_path: str) -> dict:
    """GPT-5-chat extraction avec OCR text + image (Bearer token auth)"""
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode()

    mime = "image/jpeg" if image_path.lower().endswith((".jpg", ".jpeg")) else "image/png"

    messages = [
        {"role": "system", "content": f"""{ARGUS_SYSTEM_PROMPT}

Output the extracted data as JSON matching this schema:
{json.dumps(ARGUS_SCHEMA, indent=2)}

Return ONLY valid JSON, no markdown formatting."""},
        {"role": "user", "content": [
            {"type": "text", "text": f"Here is the OCR text extracted from the document:\n\n{ocr_text}"},
            {"type": "image_url", "image_url": {"url": f"data:{mime};base64,{b64}"}}
        ]}
    ]

    token = get_bearer_token()
    resp = requests.post(
        GPT_FULL_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json={"messages": messages, "max_tokens": 4096, "temperature": 0},
        timeout=120,
    )
    resp.raise_for_status()

    content = resp.json()["choices"][0]["message"]["content"]
    content = content.strip()
    if content.startswith("```"):
        content = content.split("\n", 1)[1]
    if content.endswith("```"):
        content = content.rsplit("```", 1)[0]

    return json.loads(content.strip())

# --- Quick test du token ---
try:
    test_token = get_bearer_token()
    print(f"✅ Bearer token obtenu ({len(test_token)} chars)")
except Exception as e:
    print(f"❌ Erreur token: {e}")

print(f"\n✅ Pipeline ARGUS-style prêt (3 services)")
print(f"   OCR A : Document Intelligence prebuilt-layout (API key)")
print(f"   OCR B : Mistral Document AI mistral-document-ai-2505 (Bearer token / Entra ID)")
print(f"   LLM   : GPT-5-chat (Bearer token / Entra ID)")
print(f"   Schema: {len(ARGUS_SCHEMA)} champs de premier niveau")

✅ Bearer token obtenu (2283 chars)

✅ Pipeline ARGUS-style prêt (3 services)
   OCR A : Document Intelligence prebuilt-layout (API key)
   OCR B : Mistral Doc AI (API key)
   LLM   : GPT-5-chat (Bearer token / Entra ID)
   Schema: 15 champs de premier niveau


In [12]:
# === Section 5: Exécuter ARGUS-style sur les 20 premières images ===
# On limite à 20 documents (même set que Content Understanding)

DOCS_FOLDER = r"c:\Users\ayac\Downloads\archive (1)\batch_1\batch_1\batch1_1"

# Dossiers de sortie par variante OCR
ARGUS_DI_OUTPUT = r"c:\Users\ayac\Downloads\archive (1)\batch_1\batch_1\docu_results_batch1_1\argus-style-di"
ARGUS_MISTRAL_OUTPUT = r"c:\Users\ayac\Downloads\archive (1)\batch_1\batch_1\docu_results_batch1_1\argus-style-mistral"
os.makedirs(ARGUS_DI_OUTPUT, exist_ok=True)
os.makedirs(ARGUS_MISTRAL_OUTPUT, exist_ok=True)

# ⚠️ Limiter aux 20 premières images uniquement
all_images = sorted(glob.glob(os.path.join(DOCS_FOLDER, "batch1-*.jpg")))
images = all_images[:20]
print(f"📁 {len(all_images)} images totales, on traite les {len(images)} premières\n")

def run_argus_pipeline(images, ocr_func, ocr_name, output_dir):
    """Exécute le pipeline ARGUS complet avec une fonction OCR donnée."""
    print(f"\n{'='*60}")
    print(f"  🚀 Pipeline ARGUS avec OCR = {ocr_name}")
    print(f"{'='*60}")
    
    results = []
    for i, img_path in enumerate(images):
        fname = os.path.basename(img_path).replace(".jpg", "")
        out_file = os.path.join(output_dir, f"{fname}.json")
        
        # Skip si déjà traité (et pas en erreur)
        if os.path.exists(out_file):
            with open(out_file, encoding="utf-8") as fp:
                cached = json.load(fp)
            if not cached.get("error"):
                results.append(cached)
                print(f"  [{i+1:2d}/{len(images)}] {fname} → cache ✅")
                continue
            else:
                # Re-traiter les fichiers en erreur
                os.remove(out_file)
        
        print(f"  [{i+1:2d}/{len(images)}] {fname} → ", end="", flush=True)
        t0 = time.time()
        result = {"file": f"{fname}.jpg", "ocr_method": ocr_name, "error": None}
        
        try:
            # Étape 1: OCR
            print(f"{ocr_name} OCR...", end="", flush=True)
            ocr_text = ocr_func(img_path)
            result["ocr_chars"] = len(ocr_text)
            result["ocr_text_preview"] = ocr_text[:300]
            print(f"{len(ocr_text)} chars → ", end="", flush=True)
            
            # Étape 2: GPT-5-chat extraction
            print("GPT-5 extraction...", end="", flush=True)
            time.sleep(3)  # Rate limit
            extracted = gpt_extract_argus_style(ocr_text, img_path)
            result["extraction"] = extracted
            
            fields_count = sum(1 for v in extracted.values() if v is not None and v != "" and v != [])
            result["fields_with_value"] = fields_count
            result["total_fields"] = len(extracted)
            
        except Exception as e:
            result["error"] = str(e)
            result["fields_with_value"] = 0
            result["total_fields"] = 0
            print(f"❌ {e}")
        
        result["time_seconds"] = round(time.time() - t0, 2)
        
        with open(out_file, "w", encoding="utf-8") as fp:
            json.dump(result, fp, indent=2, ensure_ascii=False)
        
        results.append(result)
        if not result["error"]:
            print(f"{result['fields_with_value']} champs en {result['time_seconds']}s ✅")
    
    ok = sum(1 for r in results if not r.get("error"))
    print(f"\n  ✅ {ocr_name}: {ok}/{len(results)} réussis")
    return results

# --- Exécution variante A : Document Intelligence ---
argus_di_results = run_argus_pipeline(images, get_di_ocr_markdown, "DocIntel", ARGUS_DI_OUTPUT)

# --- Exécution variante B : Mistral Doc AI ---
argus_mistral_results = run_argus_pipeline(images, get_mistral_ocr_markdown, "Mistral", ARGUS_MISTRAL_OUTPUT)

print(f"\n{'='*60}")
print(f"  ✅ Les deux pipelines ARGUS terminés !")
print(f"     DocIntel : {sum(1 for r in argus_di_results if not r.get('error'))}/{len(argus_di_results)}")
print(f"     Mistral  : {sum(1 for r in argus_mistral_results if not r.get('error'))}/{len(argus_mistral_results)}")

📁 499 images totales, on traite les 20 premières


  🚀 Pipeline ARGUS avec OCR = DocIntel
  [ 1/20] batch1-0001 → cache ✅
  [ 2/20] batch1-0002 → cache ✅
  [ 3/20] batch1-0003 → cache ✅
  [ 4/20] batch1-0004 → cache ✅
  [ 5/20] batch1-0005 → cache ✅
  [ 6/20] batch1-0006 → cache ✅
  [ 7/20] batch1-0007 → cache ✅
  [ 8/20] batch1-0008 → cache ✅
  [ 9/20] batch1-0009 → cache ✅
  [10/20] batch1-0010 → cache ✅
  [11/20] batch1-0011 → cache ✅
  [12/20] batch1-0012 → cache ✅
  [13/20] batch1-0013 → cache ✅
  [14/20] batch1-0014 → cache ✅
  [15/20] batch1-0015 → cache ✅
  [16/20] batch1-0016 → cache ✅
  [17/20] batch1-0017 → DocIntel OCR...920 chars → GPT-5 extraction...14 champs en 22.98s ✅
  [18/20] batch1-0018 → cache ✅
  [19/20] batch1-0019 → cache ✅
  [20/20] batch1-0020 → cache ✅

  ✅ DocIntel: 20/20 réussis

  🚀 Pipeline ARGUS avec OCR = Mistral
  [ 1/20] batch1-0001 → Mistral OCR...❌ 403 Client Error: Forbidden for url: https://content-understanding--resource.services.ai.azure.com/pro

In [13]:
# === Section 6: Comparaison côte à côte — 3 méthodes ===
print(f"\n{'='*120}")
print(f"   COMPARAISON CÔTE À CÔTE : CONTENT UNDERSTANDING  vs  ARGUS+DocIntel  vs  ARGUS+Mistral")
print(f"{'='*120}")

header = (f"{'Document':<18} │ {'CU Champs':>10} │ {'CU Conf':>8} │ "
          f"{'DI Champs':>10} │ {'DI OCR':>8} │ {'DI t(s)':>7} │ "
          f"{'MI Champs':>10} │ {'MI OCR':>8} │ {'MI t(s)':>7}")
print(f"\n{header}")
print(f"{'─'*18}─┼─{'─'*10}─┼─{'─'*8}─┼─{'─'*10}─┼─{'─'*8}─┼─{'─'*7}─┼─{'─'*10}─┼─{'─'*8}─┼─{'─'*7}")

for cu_m in cu_metrics:
    fname = cu_m["file"].replace(".json", "")
    
    di_m = next((r for r in argus_di_results if r["file"].replace(".jpg", "") == fname), None)
    mi_m = next((r for r in argus_mistral_results if r["file"].replace(".jpg", "") == fname), None)
    
    cu_f = f"{cu_m['fields_with_value']}/{cu_m['total_fields']}"
    cu_c = f"{cu_m['avg_confidence']:.2f}"
    
    if di_m and not di_m.get("error"):
        di_f = f"{di_m['fields_with_value']}/{di_m['total_fields']}"
        di_o = f"{di_m.get('ocr_chars', 0):,}"
        di_t = f"{di_m.get('time_seconds', 0):.1f}"
    else:
        di_f, di_o, di_t = ("❌", "-", "-") if di_m else ("N/A", "N/A", "N/A")
    
    if mi_m and not mi_m.get("error"):
        mi_f = f"{mi_m['fields_with_value']}/{mi_m['total_fields']}"
        mi_o = f"{mi_m.get('ocr_chars', 0):,}"
        mi_t = f"{mi_m.get('time_seconds', 0):.1f}"
    else:
        mi_f, mi_o, mi_t = ("❌", "-", "-") if mi_m else ("N/A", "N/A", "N/A")
    
    print(f"{fname:<18} │ {cu_f:>10} │ {cu_c:>8} │ {di_f:>10} │ {di_o:>8} │ {di_t:>7} │ {mi_f:>10} │ {mi_o:>8} │ {mi_t:>7}")

# --- Résumé agrégé ---
print(f"\n{'='*120}")
print(f"   RÉSUMÉ AGRÉGÉ")
print(f"{'='*120}")

def summarize(results, name):
    ok = [r for r in results if not r.get("error")]
    if not ok:
        print(f"  {name}: aucun résultat")
        return
    fields = [r["fields_with_value"] for r in ok]
    ocr_chars = [r.get("ocr_chars", 0) for r in ok]
    times = [r.get("time_seconds", 0) for r in ok]
    print(f"  {name}:")
    print(f"    Réussis          : {len(ok)}/{len(results)}")
    print(f"    Champs/doc (moy) : {statistics.mean(fields):.1f}")
    print(f"    Champs/doc (min) : {min(fields)}")
    print(f"    Champs/doc (max) : {max(fields)}")
    print(f"    OCR chars (moy)  : {statistics.mean(ocr_chars):,.0f}")
    print(f"    Temps/doc (moy)  : {statistics.mean(times):.1f}s")
    print()

cu_ok = [m for m in cu_metrics if m["fields_with_value"] > 0]
print(f"  Content Understanding:")
print(f"    Réussis          : {len(cu_ok)}/{len(cu_metrics)}")
print(f"    Champs/doc (moy) : {statistics.mean([m['fields_with_value'] for m in cu_ok]):.1f}")
print(f"    Confidence (moy) : {statistics.mean([m['avg_confidence'] for m in cu_ok if m['avg_confidence'] > 0]):.3f}")
print(f"    Markdown (moy)   : {statistics.mean([m['markdown_chars'] for m in cu_metrics]):,.0f}")
print()

summarize(argus_di_results, "ARGUS + DocIntel OCR")
summarize(argus_mistral_results, "ARGUS + Mistral OCR")


   COMPARAISON CÔTE À CÔTE : CONTENT UNDERSTANDING  vs  ARGUS+DocIntel  vs  ARGUS+Mistral

Document           │  CU Champs │  CU Conf │  DI Champs │   DI OCR │ DI t(s) │  MI Champs │   MI OCR │ MI t(s)
───────────────────┼────────────┼──────────┼────────────┼──────────┼─────────┼────────────┼──────────┼────────
batch1-0001        │      17/31 │     0.82 │      14/15 │    1,203 │    30.1 │          ❌ │        - │       -
batch1-0002        │      17/31 │     0.83 │      14/15 │      976 │    27.8 │          ❌ │        - │       -
batch1-0003        │      17/31 │     0.83 │      14/15 │      812 │    18.4 │          ❌ │        - │       -
batch1-0004        │      17/31 │     0.82 │      14/15 │    1,004 │    24.9 │          ❌ │        - │       -
batch1-0005        │      17/31 │     0.81 │      14/15 │      943 │    26.6 │          ❌ │        - │       -
batch1-0006        │       0/31 │     0.72 │      14/15 │      810 │    28.8 │          ❌ │        - │       -
batch1-0007        │